In [0]:
dbutils.notebook.run("CS452_AISQL_db_setup", 0)
dbutils.notebook.run("populate_db", 0)

In [0]:
%pip install openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import openai
import os

openai.api_key = dbutils.secrets.get(scope = "openai", key = "openai_key")



In [0]:
def getChatGptResponse(content):
    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": content}],
        stream=True,
    )

    responseList = []
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            responseList.append(chunk.choices[0].delta.content)

    result = "".join(responseList)
    return result

In [0]:
schema = ""
tables = [
    row.tableName
    for row in spark.sql("SHOW TABLES IN my_league_db").collect()
]

for table in tables:
    columns = spark.sql(
        f"DESCRIBE TABLE my_league_db.{table}"
    ).select("col_name", "data_type").collect()
    col_defs = [
        f"{row.col_name} {row.data_type}"
        for row in columns
        if row.col_name not in ("#", "", None)
    ]
    schema += f"CREATE TABLE my_league_db.{table} (\n    " + ",\n    ".join(col_defs) + "\n);\n\n"

In [0]:
common_sql_only_request = (
    " Give me a Spark SQL select statement that answers the question. "
    "Only respond with Spark SQL syntax. Include my_league_db. before any table name. If there is an error do not explain it!"
)

strategies = {
    "zero_shot": schema + common_sql_only_request,
    "single_domain_double_shot": (
        schema +
        " Who are the players on Team 1? " +
        "\nSELECT p.firstName, p.lastName FROM Player pl JOIN Person p ON pl.byuId = p.byuId WHERE pl.teamId = 1;\n" +
        common_sql_only_request
    )
}

In [0]:
questions = [
    "List all teams in Division A.",
    "How many players are on each team?",
    "Show the full name and email of all referees.",
    "Which games are scheduled at Stadium A?",
    "What are the names and cities of all venues?",
    "Which players are assigned to Team 5?",
    "Show all games scheduled after July 15, 2024.",
    "List all divisions and the number of teams in each.",
    "Which referees are assigned to games on August 1, 2024?",
    "Show the away team ratings for all games in Division B."
]

In [0]:
def sanitizeForJustSql(value):
    gptStartSqlMarker = "```sql"
    gptEndSqlMarker = "```"
    if gptStartSqlMarker in value:
        value = value.split(gptStartSqlMarker)[1]
    if gptEndSqlMarker in value:
        value = value.split(gptEndSqlMarker)[0]

    return value

In [0]:
import json
from time import time
from pathlib import Path

notebook_dir = os.getcwd()

for strategy in strategies:
    responses = {"strategy": strategy, "prompt_prefix": strategies[strategy]}
    questionResults = []
    print("########################################################################")
    print(f"Running strategy: {strategy}")
    for question in questions:

        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("Question:")
        print(question)
        error = "None"
        try:
            getSqlFromQuestionEngineeredPrompt = strategies[strategy] + " " + question
            sqlSyntaxResponse = getChatGptResponse(getSqlFromQuestionEngineeredPrompt)
            sqlSyntaxResponse = sanitizeForJustSql(sqlSyntaxResponse)
            print("SQL Syntax Response:")
            print(sqlSyntaxResponse)
            df = spark.sql(sqlSyntaxResponse)
            display(df)
            queryRawResponse = str(df.collect())
            friendlyResultsPrompt = "I asked a question: \"" + question +"\" and I queried this database " + schema + " with this query " + sqlSyntaxResponse + ". The query returned the results data: \""+queryRawResponse+"\". Could you concisely answer my question using the results data?"
            friendlyResponse = getChatGptResponse(friendlyResultsPrompt)
            print("Friendly Response:")
            print(friendlyResponse)
        except Exception as err:
            error = str(err)
            print("THIS ONE PRODUCED AN ERROR!!!")

        questionResults.append({
            "question": question,
            "sql": sqlSyntaxResponse,
            "queryRawResponse": queryRawResponse,
            "friendlyResponse": friendlyResponse,
            "error": error
        })

    responses["questionResults"] = questionResults

    result_path = Path(notebook_dir) / f"response_{strategy}_{time()}.json"
    with open(result_path, "w") as outFile:
        json.dump(responses, outFile, indent=2)

print("All done!")

########################################################################
Running strategy: zero_shot
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Question:
List all teams in Division A.
SQL Syntax Response:

SELECT t.teamName
FROM my_league_db.team t
JOIN my_league_db.division d ON t.divisionId = d.divisionId
WHERE d.divisionName = 'Division A';



teamName
Team 1
Team 2
Team 3
Team 4
Team 5
Team 6
Team 7
Team 8


Friendly Response:
The teams in Division A are Team 1, Team 2, Team 3, Team 4, Team 5, Team 6, Team 7, and Team 8.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Question:
How many players are on each team?
SQL Syntax Response:

SELECT 
    my_league_db.team.teamId, 
    my_league_db.team.teamName, 
    COUNT(my_league_db.player.playerId) AS playerCount
FROM 
    my_league_db.team
LEFT JOIN 
    my_league_db.player 
ON 
    my_league_db.team.teamId = my_league_db.player.teamId
GROUP BY 
    my_league_db.team.teamId, 
    my_league_db.team.teamName



teamId,teamName,playerCount
8,Team 8,15
11,Team 11,15
20,Team 20,15
2,Team 2,15
12,Team 12,15
1,Team 1,15
10,Team 10,15
16,Team 16,15
7,Team 7,15
19,Team 19,15


Friendly Response:
Each team in the league has 15 players.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Question:
Show the full name and email of all referees.
SQL Syntax Response:

SELECT p.firstName, p.lastName, p.email
FROM my_league_db.person p
JOIN my_league_db.referee r ON p.byuId = r.byuId;



...

Friendly Response:
Certainly! Based on the query results, each division has the following number of teams:

- Division A: 8 teams
- Division B: 8 teams
- Division C: 8 teams
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Question:
Which referees are assigned to games on August 1, 2024?
SQL Syntax Response:

SELECT p.firstName, p.lastName
FROM my_league_db.referee r
JOIN my_league_db.person p ON r.byuId = p.byuId
JOIN my_league_db.game g ON r.refereeId = g.referee1Id OR r.refereeId = g.referee2Id
WHERE date_format(g.dateTime, 'yyyy-MM-dd') = '2024-08-01';



firstName,lastName
Ref1000378,Last1000378
Ref1000379,Last1000379


Friendly Response:
The referees assigned to games on August 1, 2024, are Ref1000378 Last1000378 and Ref1000379 Last1000379.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Question:
Show the away team ratings for all games in Division B.
SQL Syntax Response:

SELECT g.awayTeamRating FROM my_league_db.game g 
JOIN my_league_db.division d ON g.divisionId = d.divisionId 
WHERE d.divisionName = 'Division B';



awayTeamRating
1
4
3
1
3
2
3
1
3
3


Friendly Response:
The away team ratings for all games in Division B, based on your query, are as follows: 1, 4, 3, 1, 3, 2, 3, 1, 3, 3, 5, 5, 4, 2, 2, 1, 1, 2, 3, 5, 3, 3, 5, 2, 4, 2, 2, 3.
All done!
